<a href="https://colab.research.google.com/github/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/Beam_Selector_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Back End

### Library Import

In [173]:
import pandas as pd

### Function Definitions


#### Deformation Limit Functions

#### Stress Limit Functions

In [174]:
## Stress Limit Common Functions Definitions

def findStress(M, h, I):                         # Be careful ymax is not h/2 here as in Excel. That is why it is divided by 2 in the formula.
    _stress = (M*(h/(2*1000)))/((I*10**-8)*10**6)
    return _stress


def stressLimitCheck(stressLimit, result, M, beamLib, Iy_columNum, _yes= False, hDim_columNum=1):
    for h in range(0,len(beamLib.index)):
                Stress = findStress(M, beamLib.iat[h,hDim_columNum], beamLib.iat[h,Iy_columNum])
                if Stress < stressLimit:
                    if not _yes:
                        result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    else:
                        result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,0]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break


def stressLimitCheckLooper(M, result, stressLimit):
    for libNum in range(0,14):
        beamLib = pd.read_excel('/content/Beam_Selector_Stress_Deformation/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            stressLimitCheck(stressLimit, result, M, beamLib, 5, True)
        elif libNum == 12:
            stressLimitCheck(stressLimit, result, M, beamLib, 4, True)
        elif libNum in [7,9,10,11]:
            stressLimitCheck(stressLimit, result, M, beamLib, 3, True)
        elif libNum in range(0,7):
            stressLimitCheck(stressLimit, result, M, beamLib, 6)
    return result

#----------------------------------------------------------------------------------------------------------------------------------------------#

## All Case Stress Limit Functions Defitions

# The stress formula is same in all cases: M*y/I
# Since the bending moment is different for all cases, it is best to calculate 
# the bending moment in the function here and input it to the called function.

def case1_stress(stressLimit):
    M = force*lengthOfBeam/4 # Bending Moment
    result = ["You selected Case 1, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case2_stress(stressLimit):
    M = force*lengthOfBeam # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

# **Input Form** 

In [175]:
caseNumber = 2 #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'] {type:"raw"}
checkFor = "Stress" #@param ["Stress", "Deformation"]
force =  2000#@param {type:"number"}
lengthOfBeam =  4#@param {type:"number"}
length_a = 0 #@param {type:"number"}
length_b = 0 #@param {type:"number"}
length_c = 0 #@param {type:"number"}
youngsModulus = 203 #@param {type:"number"}
stressLimit =  100#@param {type:"number"}
deformationLimit = 0 #@param {type:"number"}


# Interface

In [176]:
def caseOne():
    if checkFor == "Stress": _result = case1_stress(stressLimit)
    return _result

def caseTwo():
    if checkFor == "Stress": _result = case2_stress(stressLimit)
    return _result

def caseCaller():
    if caseNumber == 1:
        _result = caseOne()
    elif caseNumber == 2:
        _result = caseTwo()
    else:
        _result = "Invalid"
    return _result


# **RESULT**

In [ ]:
result = caseCaller()
print(*result,sep='\n')